In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
dataset_path = '/content/drive/My Drive/coughing'

Required Libraries

In [3]:
pip install numpy pandas librosa matplotlib tensorflow scikit-learn

In [4]:
pip install pydub

In [5]:
import os
import numpy as np
import librosa
import librosa.display
import matplotlib.pyplot as plt
import soundfile as sf
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from pathlib import Path
from tensorflow.keras.utils import to_categorical

In [6]:
import os
from pathlib import Path

try:
    from pydub import AudioSegment
except ModuleNotFoundError:
    !pip install pydub
    from pydub import AudioSegment

# Function to convert mp3 to wav and count errors
def convert_mp3_to_wav(mp3_file, output_folder):
    try:
        # Load mp3 file
        audio = AudioSegment.from_mp3(mp3_file)

        # Define output wav file path
        wav_file = os.path.join(output_folder, Path(mp3_file).stem + '.wav')

        # Export as wav
        audio.export(wav_file, format="wav")

        print(f"Converted {mp3_file} to {wav_file}")
        return False  # No error
    except Exception as e:
        print(f"Error converting {mp3_file}: {e}")
        return True  # Error occurred

# Function to recursively find mp3 files and convert them, counting errors
def count_files_with_errors(folder_path, output_folder):
    error_count = 0
    files_with_errors = []

    for root, _, files in os.walk(folder_path):
        for file_name in files:
            if file_name.lower().endswith('.mp3'):
                mp3_file = os.path.join(root, file_name)
                if convert_mp3_to_wav(mp3_file, output_folder):
                    error_count += 1
                    files_with_errors.append(mp3_file)

    print(f"Total files with .wav conversion problems: {error_count}")
    if error_count > 0:
        print("Files with errors:")
        for file in files_with_errors:
            print(file)

# Main function
def main():
    folder_path = '/content/drive/MyDrive/coughing'  # Update to the correct path
    output_folder = '/content/drive/MyDrive/coughing/wav_files'  # Output folder for wav files

    # Create output folder if it doesn't exist
    Path(output_folder).mkdir(parents=True, exist_ok=True)

    # Count files with conversion errors
    count_files_with_errors(folder_path, output_folder)

if __name__ == "__main__":
    main()


Converted /content/drive/MyDrive/coughing/mild_coughing/mild_coughing_3-01.mp3 to /content/drive/MyDrive/coughing/wav_files/mild_coughing_3-01.wav
Converted /content/drive/MyDrive/coughing/mild_coughing/mild_coughing_2-01.mp3 to /content/drive/MyDrive/coughing/wav_files/mild_coughing_2-01.wav
Converted /content/drive/MyDrive/coughing/mild_coughing/mild_coughing_13-03.mp3 to /content/drive/MyDrive/coughing/wav_files/mild_coughing_13-03.wav
Converted /content/drive/MyDrive/coughing/mild_coughing/mild_coughing_1A-03.mp3 to /content/drive/MyDrive/coughing/wav_files/mild_coughing_1A-03.wav
Converted /content/drive/MyDrive/coughing/mild_coughing/mild_coughing_1B-04.mp3 to /content/drive/MyDrive/coughing/wav_files/mild_coughing_1B-04.wav
Converted /content/drive/MyDrive/coughing/mild_coughing/mild_coughing_1C-02.mp3 to /content/drive/MyDrive/coughing/wav_files/mild_coughing_1C-02.wav
Converted /content/drive/MyDrive/coughing/mild_coughing/mild_coughing_1D-04.mp3 to /content/drive/MyDrive/coug

Define function to save spectrogram as image

In [7]:
def save_spectrogram(spectrogram, file_path):
    plt.figure(figsize=(10, 4))
    librosa.display.specshow(spectrogram, sr=22050, x_axis='time', y_axis='mel')
    plt.colorbar(format='%+2.0f dB')
    plt.title('Mel-frequency spectrogram')
    plt.tight_layout()
    plt.savefig(file_path, format='png')
    plt.close()

Define function to generate spectrograms

In [8]:
def generate_spectrograms(folder_path, spectrogram_folder):
    labels = []
    for sub_folder in ['normal_speech', 'mild_coughing', 'intense_coughing']:
        sub_folder_path = Path(folder_path) / sub_folder
        if not sub_folder_path.exists():
            print(f"Directory {sub_folder_path} does not exist, skipping.")
            continue

        for file_path in sub_folder_path.iterdir():
            if file_path.suffix.lower() == '.mp3':  # Assuming the audio files are in .wav format
                try:
                    audio, sr = librosa.load(file_path, sr=None)
                    spectrogram = librosa.feature.melspectrogram(y=audio, sr=sr)
                    log_spectrogram = librosa.power_to_db(spectrogram, ref=np.max)

                    # Save spectrogram as image
                    spectrogram_file_path = Path(spectrogram_folder) / f"{file_path.stem}.png"
                    save_spectrogram(log_spectrogram, spectrogram_file_path)

                    labels.append((spectrogram_file_path, sub_folder))
                except Exception as e:
                    print(f"Error loading {file_path}: {e}")
    return labels

Define function to create tf.data.Dataset

In [9]:
def create_dataset(spectrogram_labels, batch_size=32, img_size=(128, 128)):
    file_paths, labels = zip(*spectrogram_labels)

    # Encode labels
    label_encoder = LabelEncoder()
    encoded_labels = label_encoder.fit_transform(labels)
    categorical_labels = to_categorical(encoded_labels)

    def load_image(file_path, label):
        image = tf.io.read_file(file_path)
        image = tf.image.decode_png(image, channels=1)
        image = tf.image.resize(image, img_size)
        image = tf.image.per_image_standardization(image)
        return image, label

    dataset = tf.data.Dataset.from_tensor_slices((list(map(str, file_paths)), categorical_labels))
    dataset = dataset.map(load_image, num_parallel_calls=tf.data.experimental.AUTOTUNE)
    dataset = dataset.shuffle(len(file_paths)).batch(batch_size).prefetch(tf.data.experimental.AUTOTUNE)

    return dataset, label_encoder

Define function to load audio files and generate spectrograms

In [23]:

def load_audio_files(folder_path, spectrogram_folder):
    audio_files = []
    spectrogram_files = []

    # List audio files
    for sub_folder in ['normal_speech', 'mild_coughing', 'intense_coughing']:
        sub_folder_path = os.path.join(folder_path, sub_folder)
        print(f"Checking files in {sub_folder_path}")
        for file_name in os.listdir(sub_folder_path):
            file_path = os.path.join(sub_folder_path, file_name)
            if file_path.endswith('.wav'):
                audio_files.append(file_path)

    # List spectrogram files
    for sub_folder in ['normal_speech', 'mild_coughing', 'intense_coughing']:
        sub_folder_path = os.path.join(spectrogram_folder, sub_folder)
        print(f"Checking files in {sub_folder_path}")
        try:
            for file_name in os.listdir(sub_folder_path):
                file_path = os.path.join(sub_folder_path, file_name)
                if file_path.endswith('.png'):  # Assuming spectrograms are in .png format
                    spectrogram_files.append(file_path)
        except FileNotFoundError:
            print(f"Error: Directory '{sub_folder_path}' not found.")
            continue

    # Return data
    return audio_files, spectrogram_files

Define function to preprocess the data

In [18]:
def preprocess_data(audio_data, labels):
    # Pad/truncate spectrograms to the same size
    max_shape = (128, 128)  # Example size
    processed_data = []
    for spec in audio_data:
        if spec.shape[1] < max_shape[1]:
            spec = np.pad(spec, ((0, 0), (0, max_shape[1] - spec.shape[1])), mode='constant')
        else:
            spec = spec[:, :max_shape[1]]
        processed_data.append(spec)
    processed_data = np.array(processed_data)

    # Expand dimensions to add the channel (needed for CNN)
    processed_data = np.expand_dims(processed_data, axis=-1)

    # Encode labels
    label_encoder = LabelEncoder()
    encoded_labels = label_encoder.fit_transform(labels)
    categorical_labels = to_categorical(encoded_labels)

    return processed_data, categorical_labels, label_encoder

Define function to build the CNN model

In [19]:
def build_cnn_model(input_shape, num_classes):
    model = Sequential([
        Conv2D(32, (3, 3), activation='relu', input_shape=input_shape),
        MaxPooling2D((2, 2)),
        Conv2D(64, (3, 3), activation='relu'),
        MaxPooling2D((2, 2)),
        Conv2D(128, (3, 3), activation='relu'),
        MaxPooling2D((2, 2)),
        Flatten(),
        Dense(128, activation='relu'),
        Dense(num_classes, activation='softmax')  # Number of output classes
    ])
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    return model

Define main function to execute the entire workflow

In [24]:
def main():
    folder_path = '/content/drive/MyDrive/coughing'  # Update to your audio files folder
    spectrogram_folder = '/content/drive/MyDrive/coughing/spectrogram'  # Update to your spectrogram folder

    # Create spectrogram directories if they do not exist
    for sub_folder in ['normal_speech', 'mild_coughing', 'intense_coughing']:
        sub_folder_path = os.path.join(spectrogram_folder, sub_folder)
        if not os.path.exists(sub_folder_path):
            try:
                os.makedirs(sub_folder_path)
            except OSError as e:
                print(f"Error creating directory '{sub_folder_path}': {e}")

    try:
        audio_files, spectrogram_files = load_audio_files(folder_path, spectrogram_folder)
    except Exception as e:
        print(f"Error loading data: {e}")
        return

    if not audio_files or not spectrogram_files:
        print("Error: No valid audio or spectrogram files found.")
        return

    # Print or check loaded files for verification
    print(f"Found {len(audio_files)} audio files and {len(spectrogram_files)} spectrogram files.")
    print("Sample audio file:", audio_files[0])  # Print a sample file for verification

if __name__ == "__main__":
    main()

Checking files in /content/drive/MyDrive/coughing/normal_speech
Checking files in /content/drive/MyDrive/coughing/mild_coughing
Checking files in /content/drive/MyDrive/coughing/intense_coughing
Checking files in /content/drive/MyDrive/coughing/spectrogram/normal_speech
Checking files in /content/drive/MyDrive/coughing/spectrogram/mild_coughing
Checking files in /content/drive/MyDrive/coughing/spectrogram/intense_coughing
Error: No valid audio or spectrogram files found.
